In [1]:
import pandas as pd

In [2]:
data_2011 = pd.read_csv('Calls_for_Service_2011.csv')
data_2012 = pd.read_csv('Calls_for_Service_2012.csv')
data_2013 = pd.read_csv('Calls_for_Service_2013.csv')
data_2014 = pd.read_csv('Calls_for_Service_2014.csv')
data_2015 = pd.read_csv('Calls_for_Service_2015.csv')

In [3]:
data = pd.concat([data_2011, data_2012, data_2013, data_2014, data_2015], ignore_index=True)

In [6]:
data.head()

,NOPD_Item,Type_,TypeText,Priority,MapX,MapY,TimeCreate,TimeDispatch,TimeArrive,TimeClosed,Disposition,DispositionText,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
0,A0000111,17M,MUNICIPAL ATTTACHME,1F,3673089,533629,01/01/2011 12:00:02 AM,NaN,NaN,01/01/2011 12:00:03 AM,NAT,NECESSARY ACTION TAKEN,007XX S Dupre St,70119,1,"(29.961547427694992, -90.09455728362126)"
1,A0000211,17M,MUNICIPAL ATTTACHME,1F,3673089,533629,01/01/2011 12:00:02 AM,NaN,NaN,01/01/2011 12:00:04 AM,NAT,NECESSARY ACTION TAKEN,007XX S Dupre St,70119,1,"(29.961547427694992, -90.09455728362126)"
2,A0000311,17M,MUNICIPAL ATTTACHME,1F,3673089,533629,01/01/2011 12:00:02 AM,NaN,NaN,01/01/2011 12:00:04 AM,NAT,NECESSARY ACTION TAKEN,007XX S Dupre St,70119,1,"(29.961547427694992, -90.09455728362126)"
3,A0000411,17M,MUNICIPAL ATTTACHME,1F,3673089,533629,01/01/2011 12:00:02 AM,NaN,NaN,01/01/2011 12:00:04 AM,NAT,NECESSARY ACTION TAKEN,007XX S Dupre St,70119,1,"(29.961547427694992, -90.09455728362126)"
4,A0000511,17M,MUNICIPAL ATTTACHME,1F,3673089,533629,01/01/2011 12:00:02 AM,NaN,NaN,01/01/2011 12:00:05 AM,NAT,NECESSARY ACTION TAKEN,007XX S Dupre St,70119,1,"(29.961547427694992, -90.09455728362126)"


Q1: What fraction of calls are of the most common type?

In [5]:
float(data['Type_'].value_counts().max())/data['Type_'].value_counts().sum()

0.2453953699658411

Q2: Some calls result in an officer being dispatched to the scene, and some log an arrival time. What is the median response time (dispatch to arrival), in seconds, considering only valid (i.e. non-negative) times?

In [7]:
data_with_time = data.dropna(subset = ['TimeDispatch', 'TimeArrive'])#drop data without time info

In [8]:
respond_time = pd.to_datetime(data_with_time['TimeArrive'])-pd.to_datetime(data_with_time['TimeDispatch'])#calculate the time interval

In [10]:
data_with_time['respond_time']= respond_time

/Users/jingyan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [20]:
data_with_time =  data_with_time[data_with_time['respond_time'] >= 0]#drop the data with negative respond time

In [35]:
average_time = data_with_time['respond_time'].mean().total_seconds()
average_time 

475.457581

Q3: Work out the average (mean) response time in each district. What is the difference between the average response times of the districts with the longest and shortest times?

In [31]:

group1= data_with_time['respond_time'].groupby(data_with_time['PoliceDistrict'])

In [37]:
mean_time = group1.sum()/group1.count() ##somehow the .mean() does not work on timedelta
mean_time

PoliceDistrict
0                00:06:43.759710
1                00:06:37.465449
2                00:08:31.211433
3                00:08:25.103193
4                00:08:52.475356
5                00:07:23.965941
6                00:06:56.191877
7                00:09:47.512741
8                00:07:17.151297
Name: respond_time, dtype: timedelta64[ns]

In [38]:
mean_time.max().total_seconds()-mean_time.min().total_seconds()

190.04729200000003

Q4: We can define surprising event types as those that occur more often in a district than they do over the whole city. What is the largest ratio of the conditional probability of an event type given a district to the unconditional probably of that event type? Consider only events types which have more than 100 events. Note that some events have their locations anonymized and are reported as being in district "0". These should be ignored.

In [39]:
data_with_district = data[data['PoliceDistrict']!= 0] # discard data in district 0

In [44]:
type_count = data_with_district['Type_'].value_counts() # count 'Type_' in the whole city
type_count = type_count[type_count >= 100] # discard event type that less than 100
type_probability = type_count/type_count.sum() # calculate the event probability for each type

In [67]:
probability_df = pd.DataFrame(type_probability, columns = ['total_probability']) #make it a dataframe

In [58]:
group2 = data_with_district['Type_'].groupby(data_with_time['PoliceDistrict']) #group the "Type_" data by districts

In [68]:
#calculate the probability of each event in a district and merge them with the probability_df 
for i, group in group2:
    type_count = group.value_counts()
    probability = type_count/type_count.sum()
    count_df = pd.DataFrame(probability, columns = [int(i)])
    probability_df = probability_df.join(count_df, how = 'left')

In [72]:
# We want to quantify the ratio conditional probability of an event type given a district to the unconditional 
probability_df = probability_df.div(probability_df['total_probability'], axis = 0)
probability_df.max() # find the max ratio

total_probability    1.000000
1                    2.256177
2                    2.006965
3                    2.332470
4                    2.613202
5                    2.695849
6                    2.462182
7                    3.597843
8                    5.663850
dtype: float64

Q5: Find the call type that displayed the largest percentage decrease in volume between 2011 and 2015. What is the fraction of the 2011 volume that this decrease represents? The answer should be between 0 and 1.

In [73]:
# Here I'd work on the individule dataframe from each year. I count call type from each year and make them a dataframe
dfs = [data_2011, data_2012, data_2013, data_2014, data_2015]
def count_type(dfs, type_df):
    for i in range(len(dfs)):
        type_count1 = dfs[i]['Type_'].value_counts()
        type_df1 = pd.DataFrame(type_count1, columns = [2011+i]) 
        type_df = type_df.join(type_df1, how = 'left')
    return type_df

In [74]:
type_count = data['Type_'].value_counts()
type_df = pd.DataFrame(type_count, columns = ['total_count']) #use value_count from all data as a reference column
type_count_df = count_type(dfs, type_df)

In [75]:
#the largest percentage decrease in volume between 2011 and 2015
decrease = (type_count_df[2011]- type_count_df[2015])/type_count_df[2011]
decrease.max()

0.99444444444444446

Q6: The disposition represents the action that was taken to address the serivce call. Consider how the disposition of calls changes with the hour of the record's creation time. Find the disposition whose fraction of that hour's disposition varies the most over a typical day. What is its variation (maximum fraction minus minimum fraction)?

In [84]:
data_disposition_time = data[['Disposition', 'TimeCreate']].dropna() # delete all data contains na in 'Disposition', 'TimeCreate'

In [85]:
times = pd.to_datetime(data_disposition_time['TimeCreate'])

In [88]:
data_disposition_time['time'] = times 

In [87]:
hour= times.apply(lambda x: x.hour) # get the hour information in each timedata
data_disposition_time['hour']= hour #add the 'hour' column to the data_disposition_time dataframe

In [91]:
group3 = data_disposition_time['Disposition'].groupby(data_disposition_time['hour']) #group the Disposition type by hour

In [96]:
dis_count = data_disposition_time['Disposition'].value_counts() #calculate the overall disposition ratio as a reference
count_all = dis_count/dis_count.sum()
dis_day_df = pd.DataFrame(count_all, columns = ['overall'])

In [98]:
for i, group in group3:
    dis_count = group.value_counts()
    dis_count = dis_count/dis_count.sum()
    count_df = pd.DataFrame(dis_count, columns = [i])
    dis_day_df = dis_day_df.join(count_df, how = 'left')

In [100]:
del dis_day_df['overall']

In [102]:
variations = dis_day_df.max(axis=1)-dis_day_df.min(axis=1)

In [104]:
variations.max()

0.18808946562536122

Q6: We can use the call locations to estimate the areas of the police districts. Represent each as an ellipse with semi-axes given by a single standard deviation of the longitude and latitude. What is the area, in square kilometers, of the largest district measured in this manner?

In [105]:
#the location data is a string. So first step is to transfer it into float data 
def clean_location(location_data):
    location_data = location_data.strip('(').strip(')').split(',')
    lat = float(location_data[0])
    lon = float(location_data[1])
    return lat, lon

In [106]:
#get ride of missing data, and data in district one
data_with_location = data[['Location', 'PoliceDistrict']].dropna()
data_with_location = data_with_location[data_with_location['PoliceDistrict']!= 0]

In [115]:
data_with_location['Location'] = data_with_location['Location'].apply(lambda x: clean_location(x))# get the location data

In [118]:
locations = pd.DataFrame(data_with_location['Location'].values.tolist())

In [119]:
locations.columns = ['lat', 'lon'] #change name of locations columns
locations['PoliceDistrict']=data_with_location['PoliceDistrict'] # add district infor
locations = locations[locations['lat']>1]#get ride of werid data that is close to 0
locations = locations[locations['lon']< -1]

In [123]:
group4 = locations.groupby('PoliceDistrict')

In [125]:
#save the min/max lat/lon values in each district
location = []
for i, group in group4:
    a1 =((group['lat'].min()+ group['lat'].max())/2, group['lon'].min()) 
    a2 = ((group['lat'].min()+ group['lat'].max())/2, group['lon'].max())
    b1 = (group['lat'].min(),(group['lon'].min()+group['lon'].max())/2)
    b2 = (group['lat'].max(),(group['lon'].min()+group['lon'].max())/2)
    location.append([a1,a2,b1,b2])

In [126]:
from geopy.distance import vincenty
from math import pi
import numpy as np
#calculate an elipse area 
def calculate_area(loc1, loc2, loc3, loc4):
    a = vincenty(loc1, loc2).meters/2000
    b = vincenty(loc3, loc4).meters/2000
    return pi*a*b
areas = []
for loc in location:
    loc1 = loc[0]
    loc2 = loc[1]
    loc3 = loc[2]
    loc4 = loc[3]
    area = calculate_area(loc1, loc2, loc3, loc4)
    areas.append(area)
np.asarray(area).max() 

988.72006531349064

Q6: The calls are assigned a priority. Some types of calls will receive a greater variety of priorities. To understand which type of call has the most variation in priority, find the type of call whose most common priority is the smallest fraction of all calls of that type. What is that smallest fraction?

In [127]:
group5 = data[['Type_', 'Priority']].groupby('Type_') #group priority data by type

In [129]:
#Calculate the highest priority ratio in each group
results = []
for i, group in group5:
    ratio = float(group['Priority'].value_counts().max())/group['Priority'].value_counts().sum() 
    results.append(ratio)

In [132]:
print np.asarray(results).min()

0.3073611709
